参考: https://github.com/samn33/qlazy/blob/master/example/py/SurfaceCode/error_correction.py

## ナイーブな実装だと遅い

In [1]:
# from qlazy import QState
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)


In [2]:
class BeliefPropagation(nn.Module):
    def __init__(self, num_nodes, num_edges):
        super(BeliefPropagation, self).__init__()
        
        # 学習可能なパラメータの定義
        self.b = nn.Parameter(torch.ones(num_nodes))
        self.w = nn.Parameter(torch.ones(num_edges, num_nodes))

        # b, w を 1 に固定した場合 (BP と等価)
        # self.b = torch.ones(num_nodes)
        # self.w = torch.ones(num_edges, num_nodes)
    
    def forward(self, l_v, h, s_c, iterations):
        num_nodes = l_v.size(0)
        num_edges = s_c.size(0)
        
        # メッセージ初期化
        mu_v_to_c = torch.zeros(num_nodes, num_edges, dtype=torch.float32)
        mu_c_to_v = torch.zeros(num_edges, num_nodes, dtype=torch.float32)
        
        # メッセージ伝播ループ
        for t in range(iterations):
            # メッセージ伝播 v -> c
            # print(self.w)
            # print(mu_c_to_v)
            # print('mu * w')
            # print(torch.matmul(mu_c_to_v, self.w))
            # print('l_v * b_v')
            # print(l_v.unsqueeze(1).size())
            # print(self.b_v)
            # print(l_v.unsqueeze(1) * self.b_v)
            # print(torch.cat([self.b_v.unsqueeze(1), self.b_v.unsqueeze(1)], dim=1).size())
            mu_v_to_c_new  = torch.zeros(num_nodes, num_edges, dtype=torch.float32)
            print('ok')
            
            for v in range(num_nodes):
                for c in range(num_edges):
                    mu_v_to_c_new[v, c] = self.update_message_v_to_c(v, c, l_v, h, mu_c_to_v, self.w, self.b)
            print('ok')
            
            # メッセージ伝播 c -> v
            # print(s_c.unsqueeze(1))
            # print(torch.ones(num_nodes, num_edges, dtype=torch.float32))
            mu_c_to_v_new = torch.zeros(num_edges, num_nodes, dtype=torch.float32)

            for c in range(num_edges):
                # print(c)
                for v in range(num_nodes):
                    mu_c_to_v_new[c, v] = self.update_message_c_to_v(c, v, h, s_c, mu_v_to_c)
            
            mu_v_to_c = mu_v_to_c_new
            mu_c_to_v = mu_c_to_v_new
        
        # 最終的なビリーフ計算
        # print(l_v)
        # print(self.b_v)
        mu_v = torch.zeros(num_nodes, dtype=torch.float32)

        for v in range(num_nodes):
            mu_v[v] = self.merginalization(v, l_v, h, mu_c_to_v, self.w, self.b)
        
        sigma_mu_v = self.hf_sigmoid(mu_v)
        
        return sigma_mu_v
    
    def update_message_v_to_c(self, v, c, l_v, h, mu_c_to_v, w, b):
        num_edges = h.size(0)

        message = l_v[v] * b[v]
        h_row = h[:, v]

        for c_prime in range(num_edges):
            if h_row[c_prime] == 1 and c_prime != c:
                # print(c)
                # print('c_prime')
                # print(c_prime)
                # print('v')
                # print(v)
                message += mu_c_to_v[c_prime, v] * w[c_prime, v]

        return message

    def update_message_c_to_v(self, c, v, h, s_c, mu_v_to_c):
        num_nodes = h.size(1)

        message = 1
        h_line = h[c, :]

        # print(h[c, :])
        for v_prime in range(num_nodes):
            if h_line[v_prime] == 1 and v_prime != v:
                message *= torch.tanh(mu_v_to_c[v_prime, c] / 2)
        
        message = (-1) ** s_c[c] * 2 * torch.atanh(message)

        return message
    
    def merginalization(self, v, l_v, h, mu_c_to_v, w, b):
        num_edges = h.size(0)

        message = l_v[v] * b[v]
        h_row = h[:, v]
        # print('h_row')
        # print(h_row)

        for c in range(num_edges):
            if h_row[c] == 1:
                message += mu_c_to_v[c, v] * w[c, v]

        return message
    
    def hf_sigmoid(self, x):
        return 1 / (torch.exp(x) + 1)
    


In [3]:
#continue with the NBP_oc class, some tool functions
#NBP_ocクラスを継続し、いくつかのツール機能を追加する。
def load_matrices(codeType, n, k, m_oc, m1, device):
    """reads in the check matrix for decoding as well as the dual matrix for checking decoding success"""
    """デコードのためのチェックマトリックスと、デコードの成功をチェックするためのデュアルマトリックスを読み込む"""
    file_nameGx = "./PCMs/" + codeType + "_" + str(n) + "_" + str(
        k) + "/" + codeType + "_" + str(n) + "_" + str(k) + "_Gx.alist"
    file_nameGz = "./PCMs/" + codeType + "_" + str(n) + "_" + str(
        k) + "/" + codeType + "_" + str(n) + "_" + str(k) + "_Gz.alist"
    Gx = readAlist(file_nameGx)
    Gz = readAlist(file_nameGz)

    file_nameH = "./PCMs/" + codeType + "_" + str(n) + "_" + str(
        k) + "/" + codeType + "_" + str(n) + "_" + str(k) + "_H_" + str(m_oc) + ".alist"

    H = readAlist(file_nameH)
    H = torch.from_numpy(H).float().to(device)
    # self.H = H
    Hx = H[0:m1, :]
    Hz = H[m1:m_oc, :]
    # Gx = torch.from_numpy(Gx).float()
    # Gz = torch.from_numpy(Gz).float()
    # Hx = torch.from_numpy(Hx).float()
    # Hz = torch.from_numpy(Hz).float()


    # first dim for batches.
    # self.Hx = self.unsqueeze_batches(Hx).float().to(self.device)
    # self.Hz = self.unsqueeze_batches(Hz).float().to(self.device)
    # self.Gx = self.unsqueeze_batches(Gx).float().to(self.device)
    # self.Gz = self.unsqueeze_batches(Gz).float().to(self.device)

    # H = torch.cat((Hx, Hz), dim=1).float().to(device)
    # self.H_reverse = 1 - self.H

    return H, Hx, Hz


def readAlist(directory):
    '''
    Reads in a parity check matrix (pcm) in A-list format from text file. returns the pcm in form of a numpy array with 0/1 bits as float64.
    テキストファイルからAリスト形式のパリティチェック行列(pcm)を読み込み、float64として0/1ビットのnumpy配列形式でpcmを返す
    '''

    alist_raw = []
    with open(directory, "r") as f:
        lines = f.readlines()
        for line in lines:
            # remove trailing newline \n and split at spaces:
            line = line.rstrip().split(" ")
            # map string to int:
            line = list(map(int, line))
            alist_raw.append(line)
    alist_numpy = alistToNumpy(alist_raw)
    alist_numpy = alist_numpy.astype(float)
    return alist_numpy


def alistToNumpy(lines):
    '''Converts a parity-check matrix in AList format to a 0/1 numpy array'''
    nCols, nRows = lines[0]
    if len(lines[2]) == nCols and len(lines[3]) == nRows:
        startIndex = 4
    else:
        startIndex = 2
    matrix = np.zeros((nRows, nCols), dtype=float)
    for col, nonzeros in enumerate(lines[startIndex:startIndex + nCols]):
        for rowIndex in nonzeros:
            if rowIndex != 0:
                matrix[rowIndex - 1, col] = 1
    return matrix

In [4]:
# give parameters for the code and decoder
n = 46
k = 2
m = 800 #number of checks, can also use 46 or 44
m1 = m // 2
m2 = m // 2
n_iterations = 6
codeType = 'GB'
device = 'cuda' if torch.cuda.is_available() else 'cpu'


H, Hx, Hz = load_matrices(codeType, n, k, m, m1, device)

In [7]:
H.size()

torch.Size([800, 46])

In [11]:
Hx.size()

torch.Size([400, 46])

In [5]:
Hx_reverse = 1 - Hx
Hz_reverse = 1 - Hz
H_v = torch.cat([Hz_reverse, Hx_reverse], dim=0)

In [15]:
H_v.size()

torch.Size([800, 46])

In [17]:
Hz

tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [1., 1., 0.,  ..., 0., 1., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [16]:
H_v

tensor([[1., 1., 1.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        ...,
        [0., 0., 1.,  ..., 1., 0., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.]], device='cuda:0')

In [6]:
e_v = torch.randint(2, (n,)).to(device).float()

In [7]:
syndrome = torch.matmul(H, e_v)
syndrome = torch.remainder(syndrome, 2)

In [33]:
syndrome.size()

torch.Size([800])

In [8]:
# 使用例
seed = np.random.RandomState(0)

k, n = H.shape
print("Number of coded bits:", n)

iterations = 1
l_v = torch.tensor([3.74899243611] * n)

model = BeliefPropagation(num_nodes=n, num_edges=k)

# sigma_mu_v = model(l_v=l_v, h=H_tensor, s_c=syndrome_tensor, iterations=iterations)
# print(sigma_mu_v)
# exit()

optimizer = optim.Adam(model.parameters(), lr=0.01)  # 最適化手法

O = torch.zeros((n // 2, n // 2))
I = torch.eye(n // 2)

OI = torch.cat((O, I), 1)
IO = torch.cat((I, O), 1)

M = torch.cat((OI, IO), 0)

print('M:')
print(M)
print(M.dtype)

# print('Error pattern:', e_v_tensor.to(torch.int32).numpy())
# sigma_mu_v = model(l_v=l_v, h=H_tensor, s_c=s_c_tensor, iterations=iterations)
# print('sigma_mu_v:')
# print(sigma_mu_v.numpy())
# exit()

print('H_tensor')
print(H)

print('H_v_tensor')
print(H_v)

history = {'epoch': [], 'loss': []}
for epoch in tqdm(range(101)):
    optimizer.zero_grad()
    # print(l_v[2])
    sigma_mu_v = model(l_v=l_v, h=H, s_c=syndrome, iterations=iterations)

    # print('H_v * M')
    # print(torch.matmul(H_v_tensor, M).shape)

    # print('x')
    # print(torch.matmul(torch.matmul(H_v_tensor, M), e_v + sigma_mu_v).shape)

    # print(e_v)
    # print(sigma_mu_v)

    loss = torch.sum(torch.abs(torch.sin((torch.pi / 2) * (torch.matmul(torch.matmul(H_v, M), e_v + sigma_mu_v)))))
    # tqdm.write('loss: {loss}')

    # criterion = nn.BCEWithLogitsLoss()  # 二値クロスエントロピー損失関数
    # loss = criterion(sigma_mu_v, e_v)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")
        history['epoch'].append(epoch)
        history['loss'].append(loss.item())
        print(sigma_mu_v)

# 学習後のパラメータの値
# print("Learned parameters:")
# print("b_v:", model.b.data)
# print("w:", model.w.data)

print('Final loss:', loss.item())
print('Error pattern:', e_v.tolist())

decoded_message = np.where(sigma_mu_v < 0.5, 0, 1)
print("Decoded message:", decoded_message)


import matplotlib.pyplot as plt

fig = plt.figure()
fig.suptitle('Learning Curve')
ax = fig.add_subplot(111, xlabel='epoch', ylabel='loss')

ax.plot(history['epoch'], history['loss'], label = 'NBP')
ax.plot(history['epoch'], [history['loss'][0]] * 11, label = 'BP')

ax.legend()
fig.savefig('qldpc_nbp.png')

Number of coded bits: 46
M:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.float32
H_tensor
tensor([[0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [1., 1., 0.,  ..., 0., 1., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.]], device='cuda:0')
H_v_tensor
tensor([[1., 1., 1.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        ...,
        [0., 0., 1.,  ..., 1., 0., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.]], device='cuda:0')


  0%|          | 0/101 [00:00<?, ?it/s]

ok


  0%|          | 0/101 [00:03<?, ?it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ogawa/anaconda3/envs/NBP_demo/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_9366/798975249.py", line 46, in <module>
    sigma_mu_v = model(l_v=l_v, h=H, s_c=syndrome, iterations=iterations)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ogawa/anaconda3/envs/NBP_demo/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_9366/3055839713.py", line 38, in forward
    mu_v_to_c_new[v, c] = self.update_message_v_to_c(v, c, l_v, h, mu_c_to_v, self.w, self.b)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_9366/3055839713.py", line -1, in update_message_v_to_c
KeyboardInterrupt

During han

In [38]:
synsyndrome

tensor([4., 3., 3., 3., 4., 4., 5., 6., 4., 3., 3., 3., 4., 3., 6., 5., 3., 3.,
        4., 4., 3., 4., 4., 7., 2., 5., 2., 4., 4., 1., 3., 5., 5., 4., 4., 6.,
        6., 4., 4., 6., 6., 7., 3., 3., 5., 4., 4., 5., 4., 7., 6., 6., 6., 8.,
        5., 3., 4., 5., 3., 3., 5., 5., 4., 5., 5., 6., 5., 3., 5., 8., 4., 5.,
        5., 3., 4., 5., 5., 7., 7., 6., 3., 6., 5., 3., 5., 2., 5., 4., 5., 3.,
        5., 5., 5., 4., 3., 3., 4., 4., 5., 5., 6., 2., 4., 6., 4., 3., 7., 1.,
        6., 6., 5., 4., 5., 4., 6., 4., 3., 4., 4., 3., 5., 3., 4., 5., 6., 5.,
        6., 7., 5., 5., 4., 5., 3., 7., 2., 5., 3., 6., 5., 6., 4., 4., 6., 4.,
        6., 5., 4., 6., 6., 6., 8., 4., 7., 4., 6., 6., 7., 4., 4., 6., 7., 7.,
        5., 4., 3., 5., 5., 7., 5., 7., 4., 5., 5., 7., 7., 6., 6., 6., 4., 8.,
        5., 3., 4., 2., 4., 2., 2., 4., 5., 3., 6., 5., 4., 6., 5., 5., 5., 4.,
        6., 6., 7., 5., 5., 6., 6., 4., 8., 4., 4., 6., 6., 3., 3., 3., 6., 2.,
        4., 3., 4., 4., 5., 5., 5., 3., 

In [9]:
model.device

AttributeError: 'BeliefPropagation' object has no attribute 'device'

In [10]:
model.to(device)

BeliefPropagation()

In [11]:
model.device

AttributeError: 'BeliefPropagation' object has no attribute 'device'

In [12]:
# モデルのパラメータのデバイスを確認
for name, param in model.named_parameters():
    print(f"Parameter: {name}, Device: {param.device}")

Parameter: b, Device: cuda:0
Parameter: w, Device: cuda:0
